# Data Migration Notebook Final
For the demo purpose of Property Tree, we are preparing csv for `Gavin Xu` profile and will be using to give the csv to Property Tree.

### We will be loading all the csv from the export to get the properties we will be looling at

In [1]:
#Importing the libraries
import pandas as pd
properties_before = pd.read_excel('Data Before/Properties_before.xlsx')
tenancies_rps = pd.read_excel('Data Before/Tenancies.xlsx')
contacts_rps = pd.read_excel('Data Before/Contacts.xlsx')
companies_rps = pd.read_excel('Data Before/Companies.xlsx')
landlord_rps = pd.read_excel('Data Before/Landlord.xlsx')

## 1. Preparing the properties

In [2]:
"""
Function to call the properties using API from RPS 
Link : https://webservice.reapit.net.au/lng/rest/_doc/?ApiKey=c242c818604a72582f383ba23b2853d1&ClearCache=T#/Landlord/get_properties__ID__inspections

"""

import requests
from pandas.io.json import json_normalize

def get_property_details(prop_id):
    url = f"https://webservice.reapit.net.au/lng/rest/properties/rps_lng-{prop_id}"
    headers = {
        'ApiKey': "c242c818604a72582f383ba23b2853d1",
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None


In [ ]:
# Creating a dataframe for Properties

df_properties = pd.DataFrame()
for prop_id in properties_before[properties_before['Agent Name'] == 'Kasey Neuendorf']['Reference'].tolist():
    result = get_property_details(prop_id)
    
    if result is not None:
        # Convert the result to a DataFrame
        df = json_normalize(result['Response'], sep='_')
        
        # Append the DataFrame to df_result
        df_properties = df_properties.append(df, ignore_index=True)

In [10]:
df_properties['Type']

0      [House, Townhouse]
1             [Townhouse]
2        [Unit/Apartment]
3        [Unit/Apartment]
4        [Unit/Apartment]
              ...        
123      [Unit/Apartment]
124                   NaN
125               [House]
126      [Unit/Apartment]
127                   NaN
Name: Type, Length: 128, dtype: object

### 1.1 Generating CSV for properties

In [4]:
df_properties_mapping = {
    'property_id': 'ID',
    'ownership_id': 'Landlord_ID',
    'property_type': 'Type',
    'unit': 'HouseNumber',
    'street_no': 'HouseNumber',
    'street_1': 'Address1',
    'street_2': '',
    'suburb': 'Address3',
    'state': 'Address4',
    'postcode': 'Postcode',
    'bedrooms': 'Bedroom',
    'bathrooms': 'Bathroom',
    'car_spaces': 'CarSpace',
    'lot_no': '',
    'area': 'Size',
    'authority_start': 'DateMarketed',
    'authority_end': '',
    'gained': 'TimeAmended',
    'gained_reason': '',
    'rent_account': '',
    'rent_description': '',
    'rent_amount': 'WeeklyRent',
    'rent_period': 'RentalPeriod',
    'outgoings_[n]_account': '',
    'outgoings_[n]_description': '',
    'outgoings_[n]_amount': '',
    'tag_[group name]': '',
    'fee_[fee id]': ''
}
pt_df_properties = pd.DataFrame()
for key,value in df_properties_mapping.items():
    if value != '':
        pt_df_properties[key] = df_properties[value]
    else:
        pt_df_properties[key] = ''

pt_df_properties['property_type'] = "Residential_House"
pt_df_properties['rent_period'] = pt_df_properties['rent_period'].replace({'month': "Monthly",'week':"Weekly"})
pt_df_properties.to_csv('PropertyTree_Properties.csv',index=False)

### 1.2 Getting Properties Transactions

In [5]:
def get_property_transactions(prop_id):
    url = f"https://webservice.reapit.net.au/lng/rest/landlord/properties/{prop_id}/transactions"
    headers = {
        'ApiKey': "c242c818604a72582f383ba23b2853d1",
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [6]:
df_properties_transactions = pd.DataFrame()
for prop_id in df_properties['ID'].tolist():
    result = get_property_transactions(prop_id)
    
    if result is not None:
        # Convert the result to a DataFrame
        df = json_normalize(result['Response'], sep='_')
        
        # Append the DataFrame to df_result
        df_properties_transactions = df_properties_transactions.append(df, ignore_index=True)
df_properties_transactions

C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\3537076894.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\3537076894.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_properties_transactions = df_properties_transactions.append(df, ignore_index=True)
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\3537076894.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\3537076894.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_properties_transactions = df_properties_transactions.append

,ID,DateTime,Type,Category,Description,IsCredit,IsDebit,Gross,Net,Balance,VAT,Unallocated
0,rps_lng-,2019-03-20T00:00:00+00:00,Opening Balance CR,Float,Opening Balance,True,False,NaN,NaN,NaN,NaN,NaN
1,rps_lng-HDO19014471,2019-03-20T00:00:00+00:00,Transfer,Rent,Rent from 21/03/2019 to 20/04/2019 for 15 Derw...,True,False,1825.00,1825.00,1825.00,NaN,NaN
2,rps_lng-HDO19014737,2019-03-20T00:00:00+00:00,Payment,Landlord Payment,Payment to Landlord (Breakdown: Stephen Wood $...,False,True,1704.55,1704.55,120.45,NaN,NaN
3,rps_lng-HDO19012024,2019-03-20T17:17:31+00:00,Invoice,Management Fee,Management fees at 6% for 21/03/2019 to 20/04/...,False,True,120.45,109.50,NaN,10.95,NaN
4,rps_lng-HDO19036519,2019-04-24T00:00:00+01:00,Transfer,Rent,Rent from 21/04/2019 to 20/05/2019 for 15 Derw...,True,False,1825.00,1825.00,1825.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
430,rps_lng-HDO23225910,2023-05-25T00:00:00+01:00,Payment,Landlord Payment,Payment to Landlord,False,True,388.95,388.95,1010.05,NaN,NaN
431,rps_lng-HDO23179978,2023-05-25T14:03:19+01:00,Invoice,Landlord Works Order,Council Rates - Instalment 4,False,True,482.05,482.05,528.00,NaN,NaN
432,rps_lng-HDO23192731,2023-05-25T14:03:19+01:00,Invoice,Landlord Works Order,Combined Elec Safety & Gas Service,False,True,528.00,480.00,NaN,48.00,NaN
433,rps_lng-HDO23271039,2023-06-23T00:00:00+01:00,Transfer,NaN,Payment: Rent guarantee invoice for period 26/...,True,False,1399.00,1399.00,1399.00,NaN,NaN


# 2. Tenancies.csv

In [7]:
import requests

def get_tenancy_details(tenancy_id):
    url = f"https://webservice.reapit.net.au/lng/rest/tenancies/rps_lng-{tenancy_id}"
    headers = {
        'ApiKey': "c242c818604a72582f383ba23b2853d1",
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None


In [8]:
df_tenancies = pd.DataFrame()
for tenancy_id in tenancies_rps[tenancies_rps['Agent name'] == 'Gavin Xu']['Reference'].tolist():
    result = get_tenancy_details(tenancy_id)
    
    if result is not None:
        # Convert the result to a DataFrame
        df = json_normalize(result['Response'], sep='_')
        
        # Append the DataFrame to df_result
        df_tenancies = df_tenancies.append(df, ignore_index=True)

df_tenancies = df_tenancies[df_tenancies['Status'].isin(['Tenancy Current', 'Tenancy Finished'])]

# Print the final DataFrame
#print(df_result)

C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\3403062876.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\3403062876.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tenancies = df_tenancies.append(df, ignore_index=True)
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\3403062876.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\3403062876.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tenancies = df_tenancies.append(df, ignore_index=True)
C:\Users\SahilSingh\AppData\Loca

### 2.2 Generating CSV for Tenancies

In [9]:
df_tenancies_mapping = {
    'tenancy_id': 'ID',
    'property_id': 'Property_ID',
    'original_lease_start': 'StartDate',
    'lease_start': 'StartDate',
    'lease_end': 'EndDate',
    'vacate_type': '',
    'notice_sent_received': '',
    'termination_date': '',
    'moving_out': '',
    'charge_to': '',
    'contact_1_id': 'Tenant_ID',
    'contact_[n]_auto_email_receipts': '',
    'contact_[n]_auto_email_invoices': '',
    'contact_[n]_enotices_no_consent': '',
    'contact_[n]_dont_send_marketing': '',
    'contact_[n]_type': '',
    'contact_[n]_account_name': '',
    'contact_[n]_account_bsb': '',
    'contact_[n]_account_no': '',
    'direct_debit_invoices': '',
    'direct_debit_rent': '',
    'direct_debit_contact_id': '',
    'direct_debit_max_amount': '',
    'direct_debit_schedule_description': '',
    'direct_debit_schedule_period': '',
    'direct_debit_schedule_amount': '',
    'direct_debit_schedule_start': '',
    'direct_debit_schedule_end': '',
    'last_rent_review': 'LastRentReview_DateTime',
    'next_rent_review': 'NextRentReview_DateTime',
    'rent_account': '',
    'rent_description': '',
    'rent_start': '',
    'rent_period': 'RentPeriod',
    'rent_amount': 'WeeklyRent',
    'new_rent_start': '',
    'new_rent_period': '',
    'new_rent_amount': '',
    'rent_paid_to': '',
    'rent_credit': '',
    'outgoings_[n]_account': '',
    'outgoings_[n]_description': '',
    'outgoings_[n]_amount': '',
    'outgoings_[n]_paid_to': '',
    'outgoings_[n]_credit': '',
    'new_outgoings_[n]_amount': '',
    'bond_[n]_required': '',
    'bond_[n]_account': '',
    'bond_[n]_reference': '',
    'bond_[n]_lodged_direct': '',
    'held_for_tenancy': '',
    'tag_[group name]': '',
    'fee_[fee id]': ''
}
pt_df_tenancies = pd.DataFrame()

for key, value in df_tenancies_mapping.items():
    if value != '':
        pt_df_tenancies[key] = df_tenancies[value]
    else:
        pt_df_tenancies[key] = ''

for index,row in df_tenancies.iterrows():
    if str(row['JointTenant']) != 'nan':
        for i in range(len(row['JointTenant'])):
            pt_df_tenancies.loc[index,f"contact_{i+2}_id"] = row['JointTenant'][i]['ID']

pt_df_tenancies['rent_period'] = pt_df_tenancies['rent_period'].replace({'month': 'Monthly', 'week': 'Weekly'})

pt_df_tenancies.to_csv("PropertyTree_Tenancies.csv",index = False)

### 2.3 Getting the Rent reviews for tenancies

In [10]:
import requests

def get_tenancies_rent_reviews(tenancy_id):
    url = f"https://webservice.reapit.net.au/lng/rest/tenancies/{tenancy_id}/rentReviews"
    headers = {
        'ApiKey': "c242c818604a72582f383ba23b2853d1",
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None


In [ ]:
df_tenancies_rent_reviews = pd.DataFrame()
for tenancy_id in df_tenancies['ID'].tolist():
    result = get_tenancies_rent_reviews(tenancy_id)
    
    if result is not None:
        # Convert the result to a DataFrame
        df = json_normalize(result['Response'], sep='_')
        
        # Append the DataFrame to df_result
        df_tenancies_rent_reviews = df_tenancies_rent_reviews.append(df, ignore_index=True)
        
# Only confirmed rent reviview
df_tenancies_rent_reviews = df_tenancies_rent_reviews[df_tenancies_rent_reviews['Confirmed'] == True]

# 3. Owners.csv

In [12]:

def get_landlord_details(prop_id):
    url = f"https://webservice.reapit.net.au/lng/rest/properties/{prop_id}/landlord"
    headers = {
        'ApiKey': "c242c818604a72582f383ba23b2853d1",
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [13]:
df_landlords = pd.DataFrame()
for prop_id in df_properties['ID'].tolist():
    result = get_landlord_details(prop_id)
    
    if result is not None:
        # Convert the result to a DataFrame
        df = json_normalize(result['Response'], sep='_')
        
        # Append the DataFrame to df_result
        df_landlords = df_landlords.append(df, ignore_index=True)

C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\3839951962.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\3839951962.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_landlords = df_landlords.append(df, ignore_index=True)
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\3839951962.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\3839951962.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_landlords = df_landlords.append(df, ignore_index=True)
C:\Users\SahilSingh\AppData\Loca

In [14]:
# Append the columns which contains contacts in the owners ID
for index,row in df_landlords.iterrows():
    for i in range(len(row['Ownership'])):
        df_landlords.loc[index,f"contact_{i+1}_id"] = row['Ownership'][i]['Contact']['ID']

#df_landlords['Ownership'].iloc[0]

# 4. Generating Contacts since for the owners csv we will use properties from contacts

In [15]:
#Appending the IDs for Joint Tenants

contacts_id_list = []

# To get the IDs of Joint contact

for value in df_tenancies['JointTenant']:
    if str(value) != 'nan':  # Check if the value is not NaN
        ids = [entry['ID'] for entry in value]  # Extract the IDs from the list of dictionaries
        contacts_id_list.extend(ids)  # Append the IDs to the list



In [16]:
# Appending the Tenants ID

ids = [value for value in df_tenancies['Tenant_ID'] if str(value) != 'nan']  # Extract the IDs from the list of dictionaries
contacts_id_list.extend(ids)  # Append the IDs to the list

In [17]:
# Appeding the landlords contacts

for index, row in df_landlords.iterrows():
    
    contacts_id_list.extend([x["ID"] for x in row['Contact']])
    
contacts_id_list = list(set(contacts_id_list))

In [18]:
def get_contact_details(contact_id):
    url = f"https://webservice.reapit.net.au/lng/rest/contacts/{contact_id}?IncludeArchive=true"
    headers = {
        'ApiKey': "c242c818604a72582f383ba23b2853d1",
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None


In [19]:
df_contacts = pd.DataFrame()
for contact_id in contacts_id_list:
    result = get_contact_details(contact_id)
    
    if result is not None:
        # Convert the result to a DataFrame
        df = json_normalize(result['Response'], sep='_')
        
        # Append the DataFrame to df_result
        df_contacts = df_contacts.append(df, ignore_index=True)

C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\2769643102.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\2769643102.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_contacts = df_contacts.append(df, ignore_index=True)
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\2769643102.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\2769643102.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_contacts = df_contacts.append(df, ignore_index=True)
C:\Users\SahilSingh\AppData\Local\Te

### 4.2 Generating CSV for contacts

In [20]:
df_contacts_mapping = {
    'contact_id': 'ID',
    'title': 'Title',
    'given_name': 'Initials',
    'family_name': 'Surname',
    'preferred_name': '',
    'address_unit': 'HouseNumber',
    'address_street_no': '',
    'address_street_1': 'Address1',
    'address_street_2': '',
    'address_suburb': 'Address3',
    'address_state': 'Address4',
    'address_postcode': 'Postcode',
    'address_country': 'Country',
    'mailing_address_unit': '',
    'mailing_address_street_no': '',
    'mailing_address_street_1': '',
    'mailing_address_street_2': '',
    'mailing_address_suburb': '',
    'mailing_address_state': '',
    'mailing_address_postcode': '',
    'mailing_address_country': '',
    'preferred_email': 'Email',
    'preferred_phone': 'Mobile',
    'additional_emails': '',
    'additional_phones': '',
    'other_contact_info': ''
}
pt_df_contacts = pd.DataFrame()
for key,value in df_contacts_mapping.items():
    if value != '':
        pt_df_contacts[key] = df_contacts[value]
    else:
        pt_df_contacts[key] = ''
pt_df_contacts.loc[pt_df_contacts['preferred_phone'].notnull(), 'preferred_phone'] = 'Mobile ' + pt_df_contacts['preferred_phone'].astype(str)
def combine_values_contacts(row):
    values = []
    if pd.notnull(row['WorkPhone']):
        values.append('Work ' + str(row['WorkPhone']))
    if pd.notnull(row['Fax']):
        values.append('Fax ' + str(row['Fax']))
    if pd.notnull(row['HomePhone']):
        values.append('Home ' + str(row['HomePhone']))
    return ' '.join(values)

pt_df_contacts['additional_phones'] = df_contacts.apply(combine_values_contacts, axis=1)

pt_df_contacts = pt_df_contacts.dropna(subset=['given_name'])
pt_df_contacts.to_csv('PropertyTree_Contacts.csv',index = False)

### 4.3 Generating owners.csv

In [21]:
df_landlords_mapping = {
    'ownership_id': 'ID',
    'name': '',
    'abn': '',
    'gst_registered': '',
    'withhold_type': '',
    'withhold_amount': '',
    'withhold_reason': '',
    'increase_withhold_amount': '',
    'contact_1_id': 'contact_1_id',
    'contact_2_id' : 'contact_2_id',
    'contact_1_create_statements': '',
    'contact_1_send_statement_by': '',
    'contact_2_send_statement_by': '',
    'contact_1_statement_style': '',
    'contact_2_statement_style': '',
    'contact_1_type': '',
    'contact_2_type': '',
    'contact_[n]_payment_method': '',
    'contact_[n]_cheque_pay_to': '',
    'contact_[n]_account_name': '',
    'contact_[n]_account_bsb': '',
    'contact_[n]_account_no': '',
    'contact_[n]_eft_reference': '',
    'contact_[n]_payment_amount': '',
    'tag_[group name]': '',
    'fee_[fee id]': ''
}
pt_df_owners = pd.DataFrame()
for key,value in df_landlords_mapping.items():
    if value != '':
        pt_df_owners[key] = df_landlords[value]
    else:
        pt_df_owners[key] = ''


In [22]:
for index,row in pt_df_owners.iterrows():

    pt_df_owners.loc[index,'name'] = df_contacts[df_contacts['ID'] == row['contact_1_id']]["Initials"].iloc[0] +" " + df_contacts[df_contacts['ID'] == row['contact_1_id']]["Surname"].iloc[0]
    pt_df_owners.loc[index,'contact_1_create_statements'] = df_contacts[df_contacts['ID'] == row['contact_1_id']]["MarketingOptIn"].iloc[0]

for index, row in pt_df_owners.iterrows():
    if row['contact_1_create_statements']:
        row['contact_1_send_statement_by'] = 'Email'
        row['contact_1_statement_style'] = 'Basic'
        row['contact_1_type'] = 'Owner'

pt_df_owners.to_csv("PropertyTree_Owners.csv",index=False)

### 4.4 Generating Owners transactions

Seems like properties and owners transactions are same

In [23]:
def get_landlord_transactions(landlord_id):
    url = f"https://webservice.reapit.net.au/lng/rest/landlords/{landlord_id}/transactions"
    headers = {
        'ApiKey': "c242c818604a72582f383ba23b2853d1",
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [24]:
df_landlords_transactions = pd.DataFrame()
for landlord_id in df_landlords['ID'].tolist():
    result = get_landlord_transactions(landlord_id)
    
    if result is not None:
        # Convert the result to a DataFrame
        df = json_normalize(result['Response'], sep='_')
        
        # Append the DataFrame to df_result
        df_landlords_transactions = df_landlords_transactions.append(df, ignore_index=True)

C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\2152325667.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\2152325667.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_landlords_transactions = df_landlords_transactions.append(df, ignore_index=True)
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\2152325667.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\2152325667.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_landlords_transactions = df_landlords_transactions.append(df,

# 5. Inspections

In [25]:
def get_tenancies_inspection(tenancy_id):
    url = f"https://webservice.reapit.net.au/lng/rest/tenancies/{tenancy_id}/inspections"
    headers = {
        'ApiKey': "c242c818604a72582f383ba23b2853d1",
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None


In [ ]:
df_tenancies_inspection = pd.DataFrame()
for tenancy_id in df_tenancies['ID'].tolist():
    result = get_tenancies_inspection(tenancy_id)
    
    if result is not None:
        # Convert the result to a DataFrame
        df = json_normalize(result['Response'], sep='_')
        df['Tenancy_ID'] = tenancy_id
        
        # Append the DataFrame to df_result
        df_tenancies_inspection = df_tenancies_inspection.append(df, ignore_index=True)

### 5.2 Generating CSV for Inspections

In [29]:
import datetime as dt

In [65]:
pd.to_datetime(pd.to_timedelta(new_df['end_time'], unit='minutes'))


TypeError: dtype timedelta64[ns] cannot be converted to datetime64[ns]

In [49]:
df_tenancies_inspection_mapping = {
    'property_id': 'Property_ID',
    'tenancy_id': 'Tenancy_ID',
    'status': 'Confirmed',
    'inspection_type': '', # Have 'Ingoing' as value
    'inspection_date': 'DateTime',
    'start_time': 'DateTime',
    'end_time': 'Duration',
    'general_note': 'FollowUpNotes',
    'owner_note': '',
    'owner_followup_item': '',
    'tenancy_note': '',
    'tenancy_action': ''
}

# Assuming the existing DataFrame is named 'existing_df'
existing_columns = df_tenancies_inspection.columns

# Create a new dictionary with non-empty values from df_tenancies_inspection_mapping
new_columns_dict = {key: value for key, value in df_tenancies_inspection_mapping.items() if value != ''}

# Create a new DataFrame with columns from the dictionary
new_df = df_tenancies_inspection[list(new_columns_dict.values())]

# Rename the columns with keys from the dictionary
new_df.columns = list(new_columns_dict.keys())


new_df['inspection_date'] = pd.to_datetime(new_df['inspection_date'], format='%Y-%m-%d')
new_df['start_time'] = pd.to_datetime(new_df['start_time'], format='%Y-%m-%d')

try:
    new_df['inspection_date'] = pd.to_datetime(new_df['inspection_date'], utc=True).dt.date
except pd._libs.tslibs.np_datetime.OutOfBoundsDatetime:
    print("Error: Invalid or out-of-range date values encountered.")

new_df['start_time'] = pd.to_datetime(new_df['start_time'], utc=True).dt.time

new_df['end_time'] = new_df['start_time'] + pd.to_timedelta(new_df['end_time'], unit='m')

import numpy as np
new_df['status'] = np.where(new_df['status'], 'Confirmed', 'Closed')

new_df = new_df.insert(3, 'inspection_type', 'Routine')

new_df.to_csv("PropertyTree_Inspections.csv", index=False)

C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\4124614079.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['inspection_date'] = pd.to_datetime(new_df['inspection_date'], format='%Y-%m-%d')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\4124614079.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['start_time'] = pd.to_datetime(new_df['start_time'], format='%Y-%m-%d')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\4124614079.py:33: SettingWithCopyWarning: 
A value 

TypeError: unsupported operand type(s) for +: 'Timedelta' and 'datetime.time'

# 6. Maintenance.csv

In [39]:
def get_tenancies_workorders(tenancy_id):
    url = f"https://webservice.reapit.net.au/lng/rest/tenancies/{tenancy_id}/worksOrders"
    headers = {
        'ApiKey': "c242c818604a72582f383ba23b2853d1",
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None


In [40]:
df_tenancies_workorders = pd.DataFrame()
for tenancy_id in df_tenancies['ID'].tolist():
    result = get_tenancies_workorders(tenancy_id)
    
    if result is not None:
        # Convert the result to a DataFrame
        df = json_normalize(result['Response'], sep='_')
        
        # Append the DataFrame to df_result
        df_tenancies_workorders = df_tenancies_workorders.append(df, ignore_index=True)


Error: 404 - {"error":[{"message":"No works order found"}]}
Error: 404 - {"error":[{"message":"No works order found"}]}
Error: 404 - {"error":[{"message":"No works order found"}]}


C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tenancies_workorders = df_tenancies_workorders.append(df, ignore_index=True)
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tenancies_workorders = df_tenancies_workorders.append(df, ignore_inde

Error: 404 - {"error":[{"message":"No works order found"}]}
Error: 404 - {"error":[{"message":"No works order found"}]}


C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tenancies_workorders = df_tenancies_workorders.append(df, ignore_index=True)


Error: 404 - {"error":[{"message":"No works order found"}]}


...

C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tenancies_workorders = df_tenancies_workorders.append(df, ignore_index=True)
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tenancies_workorders = df_tenancies_workorders.append(df, ignore_inde

Error: 404 - {"error":[{"message":"No works order found"}]}


C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tenancies_workorders = df_tenancies_workorders.append(df, ignore_index=True)
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tenancies_workorders = df_tenancies_workorders.append(df, ignore_inde

Error: 404 - {"error":[{"message":"No works order found"}]}


C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tenancies_workorders = df_tenancies_workorders.append(df, ignore_index=True)
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\446644657.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tenancies_workorders = df_tenancies_workorders.append(df, ignore_inde

### 6.1 Generating CSV for maintenance

In [41]:
df_tenancies_workorders_mapping = {
    'property_id': 'Property_ID',
    'requested_by': 'Reporter',
    'access_contact': 'Reporter',
    'tenancy_id': 'Tenancy_ID',
    'summary': 'Description',
    'details': 'Description',
    'status': 'Status',
    'quote_request_creditor_ids': 'Contractor_ID',
    'work_order_creditor_id': 'Contractor_ID'
}
pt_df_tenancies_workorders = pd.DataFrame()
for key,value in df_tenancies_workorders_mapping.items():
    pt_df_tenancies_workorders[key] = df_tenancies_workorders[value]



pt_df_tenancies_workorders['requested_by'] = pt_df_tenancies_workorders['requested_by'].fillna('Agency').replace({
    'Tenant': 'Tenancy',
    'Other': 'Agency',
    'Landlord': 'Owner'
})
pt_df_tenancies_workorders['access_contact'] = pt_df_tenancies_workorders['requested_by']
pt_df_tenancies_workorders.loc[pt_df_tenancies_workorders['requested_by'] != 'Tenancy', 'tenancy_id'] = None

pt_df_tenancies_workorders['status'] = pt_df_tenancies_workorders['status'].replace({
    'Raised': 'Open',
    'Pending quote': 'QuoteRequest',
    'Pending approval': 'OwnerAttending',
    'Completed': 'WorkOrder'
})
pt_df_tenancies_workorders.loc[pt_df_tenancies_workorders['status'] != 'QuoteRequest', 'quote_request_creditor_ids'] = None
pt_df_tenancies_workorders.loc[pt_df_tenancies_workorders['status'] != 'WorkOrder', 'work_order_creditor_id'] = None


pt_df_tenancies_workorders.to_csv("PropertyTree_Maintenance.csv", index=False)

# 7. Creditors.csv

In [42]:

df_creditors = pd.DataFrame()
for contact_id in list(set(df_tenancies_workorders['Contractor_ID'].tolist())):
    if str(contact_id) != 'nan':
        result = get_contact_details(contact_id)
        
        if result is not None:
            # Convert the result to a DataFrame
            df = json_normalize(result['Response'], sep='_')
            
            # Append the DataFrame to df_result
            df_creditors = df_creditors.append(df, ignore_index=True)

C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\4262800127.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\4262800127.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_creditors = df_creditors.append(df, ignore_index=True)
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\4262800127.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(result['Response'], sep='_')
C:\Users\SahilSingh\AppData\Local\Temp\ipykernel_51324\4262800127.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_creditors = df_creditors.append(df, ignore_index=True)
C:\Users\SahilSingh\AppData\Loca

In [43]:
df_creditors.columns

Index(['ID', 'CompanyName', 'Postcode', 'Country', 'WorkPhone', 'Email',
       'MarketingOptOut', 'MarketingOptIn', 'MarketingConsentAsked',
       'IsVendor', 'IsLandlord', 'IsTenant', 'IsTrackerUser', 'IsArchived',
       'IsApplicant', 'TimeAmended', 'TimeRegistered', 'IsErased', 'Office_ID',
       'Negotiator_ID', 'Salutation', 'HouseNumber', 'Address1', 'Address3',
       'Address4', 'Fax', 'Mobile', 'Address2', 'HouseName', 'TimeArchived',
       'DateArchived', 'Surname', 'HomePhone'],
      dtype='object')

### 7.1 Generating CSV for Creditors

In [44]:
df_creditors_mapping = {
    'creditor_id': 'ID',
    'name': 'Surname',
    'company_name': 'CompanyName',
    'abn': '',
    'address_unit': 'HouseNumber',
    'address_street_no': '',
    'address_street_1': 'Address1',
    'address_street_2': 'Address2',
    'address_suburb': 'Address3',
    'address_state': 'Address4',
    'address_postcode': 'Postcode',
    'mailing_address_unit': '',
    'mailing_address_street_no': '',
    'mailing_address_street_1': '',
    'mailing_address_street_2': '',
    'mailing_address_suburb': '',
    'mailing_address_state': '',
    'mailing_address_postcode': '',
    'preferred_email': 'Email',
    'preferred_phone': 'WorkPhone',
    'additional_emails': '',
    'additional_phones': 'Mobile',
    'other_contact_info': '',
    'preferred_comm_method': 'Email',
    'default_account_code': '',
    'is_gst_exempt': '',
    'is_priority': '',
    'is_strata_creditor': '',
    'strata_plan_number': '',
    'strata_building_name': '',
    'strata_self_managed': '',
    'strata_building_street_no': '',
    'strata_building_street_1': '',
    'strata_building_street_2': '',
    'strata_building_suburb': '',
    'strata_building_state': '',
    'strata_building_postcode': '',
    'payment_method': '',
    'cheque_pay_to': '',
    'eft_account_name': '',
    'eft_account_bsb': '',
    'eft_account_no': '',
    'bpay_reference': '',
    'contact_[n]_id': '',
    'contact_[n]_type': '',
    'tag_[group name]': ''
}
pt_df_creditors = pd.DataFrame()
for key,value in df_creditors_mapping.items():
    if value != '':
        pt_df_creditors[key] = df_creditors[value]
    else:
        pt_df_creditors[key] = ''

pt_df_creditors.loc[pt_df_creditors['preferred_phone'].notnull(), 'preferred_phone'] = 'Work ' + pt_df_creditors['preferred_phone'].astype(str)

def combine_values(row):
    values = []
    if pd.notnull(row['Mobile']):
        values.append('Mobile ' + str(row['Mobile']))
    if pd.notnull(row['Fax']):
        values.append('Fax ' + str(row['Fax']))
    if pd.notnull(row['HomePhone']):
        values.append('Home ' + str(row['HomePhone']))
    return ' '.join(values)

pt_df_creditors['additional_phones'] = df_creditors.apply(combine_values, axis=1)

pt_df_creditors.to_csv("PropertyTree_Creditors.csv", index=False)

# 8. Invoices

In [45]:
df_invoices = df_tenancies_workorders[df_tenancies_workorders['Status'] == 'Completed']


### 8.1 Generating CSV for Invoices

In [46]:
#Make to Csv
df_invoices_mapping = {
    'invoice_type': 'SupplierInvoice',
    'property_id': 'Property_ID',
    'creditor_id': 'Contractor_ID',
    'tenancy_id': 'Tenancy_ID',
    'entered_date': 'OrderDate',
    'due_date': 'RequiredDate',
    'creditor_reference': '',
    'is_priority': '',
    'line_[n]_account_code': '',
    'line_[n]_description': '',
    'line_[n]_amount_due': '',
    'line_[n]_gst_due': '',
    'line_[n]_amount_paid': ''
}
pt_df_invoices = pd.DataFrame()
for key,value in df_invoices_mapping.items():
    if value != '':
        pt_df_invoices[key] = df_invoices[value]
    else:
        pt_df_invoices[key] = ''

pt_df_invoices['invoice_type'] = pt_df_invoices.apply(lambda row: 'Tenancy' if str(row['invoice_type']) == 'nan' else 'Creditor',axis = 1)


pt_df_invoices.loc[(pt_df_invoices['creditor_id'].isnull()) & (pt_df_invoices['invoice_type'] == 'Creditor'), 'invoice_type'] = 'Tenancy'

pt_df_invoices.to_csv("PropertyTree_Invoices.csv",index=False)

# 9 Extras: Generating documents

In [ ]:
df_tenancies_workorders['SupplierInvoice'].tolist()

In [ ]:

supplier_invoice_ids = []
for item in df_tenancies_workorders['SupplierInvoice'].tolist():
    if isinstance(item, list):
        for entry in item:
            supplier_invoice_ids.append(entry['ID'])


# Print the extracted IDs
print(len(supplier_invoice_ids))

In [ ]:
# To get the Vendors Documents

url = "https://webservice.reapit.net.au/lng/rest/tenant/documents/rps_lng-VAD23000333"
headers = {
    'ApiKey': "c242c818604a72582f383ba23b2853d1",
    "Accept": "application/json"
}


response = requests.get(url, headers=headers)
if response.status_code == 200:
    print(response.json())
else:
    print(f"Error: {response.status_code} - {response.text}")
    

In [ ]:
df_tenancies_transactions[df_tenancies_transactions['Type'] == 'Invoice']

In [73]:
.dtypes

tenancy_id                           object
property_id                          object
original_lease_start                 object
lease_start                          object
lease_end                            object
vacate_type                          object
notice_sent_received                 object
termination_date                     object
moving_out                           object
charge_to                            object
contact_1_id                         object
contact_[n]_auto_email_receipts      object
contact_[n]_auto_email_invoices      object
contact_[n]_enotices_no_consent      object
contact_[n]_dont_send_marketing      object
contact_[n]_type                     object
contact_[n]_account_name             object
contact_[n]_account_bsb              object
contact_[n]_account_no               object
direct_debit_invoices                object
direct_debit_rent                    object
direct_debit_contact_id              object
direct_debit_max_amount         